In [1726]:
import math
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection

In [1727]:
train = pd.read_csv('D:/JISUNG/dacon/titanic/data/train.csv')
test = pd.read_csv('D:/JISUNG/dacon/titanic/data/test.csv')

In [1728]:
y_train = train['Survived'].values
passId = test['PassengerId']
data = pd.concat((train, test))

인덱스 리셋

In [1729]:
data = data.reset_index(drop=True)

Fare 결측치를 평균으로 치환

In [1730]:
data['Fare'].fillna(data['Fare'].mean(), inplace=True)

이름에서 성만 추출

In [1731]:
data['Last_Name'] = data['Name'].apply(lambda x: str.split(x, ",")[0])

성과 요금을 기준으로 가족 여부를 판단
- 가족이 없을 경우 생존확률 0.5
- 가족이 있을 경우 자기 자신을 제외한 나머지 가족들의 생존율을 자신의 생존확률로 채택

In [1732]:
data['Probability'] = 0.5

In [1733]:
for _, group_df in data.groupby(['Last_Name', 'Fare']):
    if len(group_df) != 1:
        for idx, row in group_df.iterrows():
            probability = group_df.drop(idx)['Survived'].mean()
            if math.isnan(probability):
                data.loc[data['PassengerId'] == row['PassengerId'], 'Probability'] = 0.5
            else:
                data.loc[data['PassengerId'] == row['PassengerId'], 'Probability'] = probability

티켓 번호를 기준으로 동승자 여부를 판단
- 동승자가 없을 경우 생존확률 0.5
- (가족이 없는 승객 중) 동승자가 있을 경우 자기 자신을 제외한 나머지 동승자들의 생존율을 자신의 생존확률로 채택

In [1734]:
data['Company'] = 0

In [1735]:
for _, group_df in data.groupby('Ticket'):
    if len(group_df) != 1:
        for idx, row in group_df.iterrows():
            data.loc[data['PassengerId'] == row['PassengerId'], 'Company'] = len(group_df)
            if row['Probability'] == 0.5:
                probability = group_df.drop(idx)['Survived'].mean()
                if math.isnan(probability):
                    data.loc[data['PassengerId'] == row['PassengerId'], 'Probability'] = 0.5
                else:
                    data.loc[data['PassengerId'] == row['PassengerId'], 'Probability'] = probability

요금 4개 범주화

In [1736]:
data['Fare'] = pd.qcut(data['Fare'], 4)

In [1737]:
label = LabelEncoder()
data['Fare'] = label.fit_transform(data['Fare'])

호칭 추출

In [1738]:
Title_Dictionary = {
    'Capt':'ETC',
    'Col':'ETC',
    'Don':'Nobility',
    'Dr':'ETC',
    'Jonkheer':'Nobility',
    'Lady':'Nobility',
    'Major':'ETC',
    'Master':'Master',
    'Miss':'Miss',
    'Mlle':'Miss',
    'Mme':'Mrs',
    'Mr':'Mr',
    'Mrs':'Mrs',
    'Ms':'Miss',
    'Dona':'Mrs',
    'Rev':'ETC',
    'Sir':'Nobility',
    'the Countess':'Nobility'
}

In [1739]:
# Title_Dictionary = {
#     'Capt':'ETC',
#     'Col':'ETC',
#     'Don':'ETC',
#     'Dr':'ETC',
#     'Jonkheer':'ETC',
#     'Lady':'ETC',
#     'Major':'ETC',
#     'Master':'Master',
#     'Miss':'Miss',
#     'Mlle':'Miss',
#     'Mme':'Mrs',
#     'Mr':'Mr',
#     'Mrs':'Mrs',
#     'Ms':'Miss',
#     'Dona':'Mrs',
#     'Rev':'ETC',
#     'Sir':'ETC',
#     'the Countess':'ETC'
# }

In [1740]:
data['Title'] = data['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
data['Title'] = data.Title.map(Title_Dictionary)

나이의 결측치를 호칭의 중앙값으로 치환

In [1741]:
data['Age'] = data.groupby('Title')['Age'].apply(lambda x: x.fillna(round(x.median(), 1)))

나이 4개 범주화

In [1742]:
data['Age'] = pd.qcut(data['Age'], 4)

In [1743]:
label = LabelEncoder()
data['Age'] = label.fit_transform(data['Age'])

호칭 숫자로 변환

In [1744]:
data['Title'] = data['Title'].replace(['Mr', 'ETC', 'Nobility', 'Miss', 'Mrs', 'Master'], [0, 1, 2, 3, 4, 5])
# data['Title'] = data['Title'].replace(['Mr', 'ETC', 'Miss', 'Mrs', 'Master'], [0, 1, 2, 3, 4])

여자와 아이

In [1745]:
data['Women_Child'] = 0

In [1746]:
data.loc[(data['Sex'] == 'female') & (data['Age'] < 10), 'Women_Child'] = 1

성별 숫자로 변환

In [1747]:
data['Sex'] = data['Sex'].replace(['male', 'female'], [0, 1])

항구 결측치 최빈값으로 치환

In [1748]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

항구 숫자로 변환

In [1749]:
data['Embarked'] = data['Embarked'].replace(['S', 'C', 'Q'], [0, 1, 2])

객실 결측치를 N으로 치환

In [1750]:
data['Cabin'].fillna('N',inplace=True)

객실의 문자 부분 추출

In [1751]:
data['Cabin'] = data['Cabin'].map(lambda x: x[0])

객실 결측치 처리
- Pclass 1 : 3
- Pclass 2 : 2
- Pclass 3 : 0

In [1752]:
Cabin_Dictionary = {
    'A':'3',
    'B':'3',
    'C':'3',
    'D':'2',
    'E':'2',
    'F':'1',
    'G':'1',
    'T':'3',
    'N':'0'
}

In [1753]:
data['Cabin'] = data.Cabin.map(Cabin_Dictionary)

In [1754]:
data['Cabin'] = data.Cabin.astype(int)

In [1755]:
data.loc[(data.Pclass == 1) & (data.Cabin == 2), 'Cabin'] = 2

가족 컬럼 생성

In [1756]:
data['Family'] = data['SibSp'] + data['Parch'] + 1

불필요 컬럼 제거

In [1757]:
data = data.drop(['Survived', 'Name', 'SibSp', 'Parch', 'Ticket', 'Last_Name', 'PassengerId'], axis=1)

train/test 데이터 분리

In [1758]:
train = data[:891]
test = data[891:]

표준화

In [1759]:
X_train = train
X_test = test

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

5개 모델 생성

In [1760]:
rf = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
lr = LogisticRegression()
xgb = XGBClassifier()
svm = SVC(probability=True)
  
models = [rf, knn, lr, xgb, svm]
scores = []

for mod in models:
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring='accuracy', cv=10)
    scores.append(acc.mean())

In [1761]:
results = pd.DataFrame({
    'Model': [
        'Random Forest',
        'K Nearest Neighbour',
        'Logistic Regression',
        'XGBoost',
        'SVM'
    ],
    'First': scores})

acc_rank = results.sort_values(by='First', ascending=False).reset_index(drop=True)
acc_rank

,Model,First
0,SVM,0.850699
1,K Nearest Neighbour,0.844007
2,XGBoost,0.842921
3,Logistic Regression,0.841785
4,Random Forest,0.839526


변수 선택

In [1762]:
xgb_imp = pd.DataFrame({'Feature':train.columns, 'xgb importance':xgb.feature_importances_})
rf_imp = pd.DataFrame({'Feature':train.columns, 'rf importance':rf.feature_importances_})

importances = xgb_imp.merge(rf_imp, on='Feature')
importances['Average'] = importances.mean(axis=1)
importances = importances.sort_values(by='Average', ascending=False).reset_index(drop=True)

importances

,Feature,xgb importance,rf importance,Average
0,Title,0.441628,0.171471,0.306549
1,Probability,0.146293,0.127241,0.136767
2,Pclass,0.159641,0.080745,0.120193
3,Cabin,0.074810,0.063896,0.069353
4,Sex,0.013011,0.125066,0.069039
5,Company,0.042345,0.077634,0.059989
6,Women_Child,0.000000,0.106050,0.053025
7,Family,0.028584,0.076314,0.052449
8,Age,0.025832,0.073529,0.049681
9,Fare,0.036234,0.055218,0.045726


변수 제거

In [1763]:
train = train.drop(['Embarked', 'Fare', 'Age'], axis=1)
test = test.drop(['Embarked', 'Fare', 'Age'], axis=1)

X_train = train
X_test = test

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

5개 모델 생성

In [1764]:
rf = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
lr = LogisticRegression()
xgb = XGBClassifier(random_state=1)
svm = SVC(probability=True)
      
models = [rf, knn, lr, xgb, svm]
scores_v2 = []

for mod in models:
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring='accuracy', cv=10)
    scores_v2.append(acc.mean())

In [1765]:
results = pd.DataFrame({
    'Model': [
        'Random Forest',
        'K Nearest Neighbour',
        'Logistic Regression',
        'XGBoost',
        'SVM'
    ],
    'First': scores,
    'Delete feature': scores_v2})

acc_rank = results.sort_values(by='Delete feature', ascending=False).reset_index(drop=True)
acc_rank

,Model,First,Delete feature
0,SVM,0.850699,0.854057
1,Random Forest,0.839526,0.847366
2,XGBoost,0.842921,0.847366
3,Logistic Regression,0.841785,0.845144
4,K Nearest Neighbour,0.844007,0.840637


SVM

In [1709]:
C = [0.001, 0.01, 0.1, 1, 5, 10, 15, 20, 50, 100]
gamma = [0.001, 0.01, 0.1, 1]

hyperparams = {'C': C, 'gamma' : gamma}

gs_svm = GridSearchCV(
    estimator=SVC(probability=True),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_svm.fit(X_train, y_train)
print(gs_svm.best_score_)
print(gs_svm.best_estimator_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8563178708179022
SVC(C=20, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   12.2s finished


LR

In [1710]:
penalty = ['l1', 'l2']
C = np.logspace(0, 5, 10)

hyperparams = {'penalty': penalty, 'C': C}

gs_lr = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_lr.fit(X_train, y_train)
print(gs_lr.best_score_)
print(gs_lr.best_estimator_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8395016006528152
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


XGB

In [1711]:
learning_rate = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
n_estimators = [10, 25, 50, 75, 100, 250, 500, 750, 1000]

hyperparams = {
    'learning_rate': learning_rate,
    'n_estimators': n_estimators
}

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_estimator_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8484840876278952
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   41.4s finished


In [1712]:
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]
min_child_weight = [1, 2, 3, 4, 5, 6]

hyperparams = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight
}

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(
        learning_rate=0.1,
        n_estimators=250
    ),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_estimator_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8529533613709119
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:   20.8s finished


In [1713]:
gamma = [i*0.1 for i in range(0,5)]

hyperparams = {'gamma': gamma}

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(
        learning_rate=0.1,
        n_estimators=250,
        max_depth=3,
        min_child_weight=3
    ),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy')

gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_estimator_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8540644027368025
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.30000000000000004,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    1.2s finished


In [1714]:
subsample = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
colsample_bytree = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    
hyperparams = {
    'subsample': subsample,
    'colsample_bytree': colsample_bytree
}

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(
        learning_rate=0.1,
        n_estimators=250,
        max_depth=3,
        min_child_weight=3,
        gamma=0.30000000000000004
    ),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_estimator_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8597012114744838
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              gamma=0.30000000000000004, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=250, n_jobs=0,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)


[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:   24.8s finished


In [1715]:
reg_alpha = [1e-5, 1e-2, 0.1, 1, 100]
    
hyperparams = {'reg_alpha': reg_alpha}

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(
        learning_rate=0.1,
        n_estimators=250,
        max_depth=3,
        min_child_weight=3,
        gamma=0.30000000000000004,
        subsample=0.9,
        colsample_bytree=0.7
    ),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_estimator_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8597012114744838
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              gamma=0.30000000000000004, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=250, n_jobs=0,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    1.3s finished


KNN

In [1716]:
n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 30, 35, 40, 50]
algorithm = ['auto']
weights = ['uniform', 'distance']
leaf_size = [1, 2, 3, 4, 5, 10, 15, 20, 25, 30]
p = [1, 2, 3]

hyperparams = {
    'algorithm': algorithm,
    'weights': weights,
    'leaf_size': leaf_size,
    'n_neighbors': n_neighbors,
    'p': p,
}

gs_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_knn.fit(X_train, y_train)
print(gs_knn.best_score_)
print(gs_knn.best_estimator_)

Fitting 5 folds for each of 1140 candidates, totalling 5700 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8563367020274935
KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')


[Parallel(n_jobs=1)]: Done 5700 out of 5700 | elapsed:  1.4min finished


RF

In [1717]:
n_estimators = [10, 25, 50, 75, 100]
max_depth = [None, 2]
max_features = [1, 3, 5, 7]
min_samples_split = [2, 4, 6, 8, 10]
min_samples_leaf = [2, 4, 6, 8, 10]

hyperparams = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

gs_rf = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy'
)

gs_rf.fit(X_train, y_train)
print(gs_rf.best_score_)
print(gs_rf.best_estimator_)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8596886573347561
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  6.5min finished


In [1718]:
models = [
    gs_rf.best_estimator_,
    gs_knn.best_estimator_,
    gs_lr.best_estimator_,
    gs_xgb.best_estimator_,
    gs_svm.best_estimator_
]
scores_v3 = []

for mod in models:
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring='accuracy', cv=10)
    scores_v3.append(acc.mean())

In [1719]:
results = pd.DataFrame({
    'Model': [
        'Random Forest',
        'K Nearest Neighbour',
        'Logistic Regression',
        'XGBoost',
        'SVM'
    ],
    'First': scores,
    'Delete feature': scores_v2,
    'Hyper parameters': scores_v3})

acc_rank = results.sort_values(by='Hyper parameters', ascending=False).reset_index(drop=True)
acc_rank

,Model,First,Delete feature,Hyper parameters
0,SVM,0.850699,0.852959,0.857428
1,K Nearest Neighbour,0.844007,0.841785,0.854082
2,XGBoost,0.842921,0.851898,0.852996
3,Random Forest,0.839526,0.850774,0.847366
4,Logistic Regression,0.841785,0.840662,0.840662


In [1720]:
# grid_hard = VotingClassifier(
#     estimators = [
#         ('Random Forest', rf),
#         ('Logistic Regression', lr),
#         ('XGBoost', xgb),
#         ('SVM', svm),
#         ('K Nearest Neighbour', knn)
#     ],
#     voting = 'hard'
# )

# grid_hard_cv = model_selection.cross_validate(grid_hard, X_train, y_train, cv=10)
# grid_hard.fit(X_train, y_train)

# grid_hard_cv['test_score'].mean()

In [1721]:
# grid_soft = VotingClassifier(
#     estimators = [
#         ('Random Forest', rf),
#         ('Logistic Regression', lr),
#         ('XGBoost', xgb),
#         ('SVM', svm),
#         ('K Nearest Neighbour', knn)
#     ],
#     voting = 'soft'
# )

# grid_soft_cv = model_selection.cross_validate(grid_soft, X_train, y_train, cv=10)
# grid_soft.fit(X_train, y_train)

# grid_soft_cv['test_score'].mean()

In [1722]:
# predictions = grid_hard.predict(X_test)

# submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis = 'columns')

# submission.columns = ['PassengerId', 'Survived']
# submission.to_csv('D:/JISUNG/dacon/titanic/submission.csv', header = True, index = False)

In [1725]:
predictions = gs_svm.best_estimator_.predict(X_test)

submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis = 'columns')

submission.columns = ['PassengerId', 'Survived']
submission.to_csv('D:/JISUNG/dacon/titanic/submission.csv', header = True, index = False)